# 1. Description
Data description sourced from the Kaggle competition page.

### train.csv
- `county` - An ID code for the county.
- `is_business` - Boolean for whether or not the prosumer is a business.
- `product_type` - ID code with the following mapping of codes to contract types: `{0: "Combined", 1: "Fixed", 2: "General service", 3: "Spot"}`.
- `target` - The consumption or production amount for the relevant segment for the hour. The segments are defined by the `county`, `is_business`, and `product_type`.
- `is_consumption` - Boolean for whether or not this row's target is consumption or production.
- `datetime` - The Estonian time in EET (UTC+2) / EEST (UTC+3). It describes the start of the 1-hour period on which target is given.
- `data_block_id` - All rows sharing the same `data_block_id` will be available at the same forecast time. This is a function of what information is available when forecasts are actually made, at 11 AM each morning. For example, if the forecast weather `data_block_id` for predictins made on October 31st is 100 then the historic weather `data_block_id` for October 31st will be 101 as the historic weather data is only actually available the next day.
- `row_id` - A unique identifier for the row.
- `prediction_unit_id` - A unique identifier for the `county`, `is_business`, and `product_type` combination. *New prediction units can appear or disappear in the test set*.

### gas_prices.csv
- `origin_date` - The date when the day-ahead prices became available.
- `forecast_date` - The date when the forecast prices should be relevant.
- `[lowest/highest]_price_per_mwh` - The lowest/highest price of natural gas that on the day ahead market that trading day, in Euros per megawatt hour equivalent.
- `data_block_id`

### client.csv
- `product_type`
- `county` - An ID code for the county. See `county_id_to_name_map.json` for the mapping of ID codes to county names.
- `eic_count` - The aggregated number of consumption points (EICs - European Identifier Code).
- `installed_capacity` - Installed photovoltaic solar panel capacity in kilowatts.
- `is_business` - Boolean for whether or not the prosumer is a business.
- `date`
- `data_block_id`

### electricity_prices.csv
- `origin_date`
- `forecast_date` - Represents the start of the 1-hour period when the price is valid.
- `euros_per_mwh` - The price of electricity on the day ahead markets in euros per megawatt hour.
- `data_block_id`

### forecast_weather.csv
Weather forecasts that would have been available at prediction time. Sourced from the [European Centre for Medium-Range Weather Forecasts](https://codes.ecmwf.int/grib/param-db/?filter=grib2).

- `[latitude/longitude]` - The coordinates of the weather forecast.
- `origin_datetime` - The timestamp of when the forecast was generated.
- `hours_ahead` - The number of hours between the forecast generation and the forecast weather. Each forecast covers 48 hours in total.
- `temperature` - The air temperature at 2 meters above ground in degrees Celsius. Estimated for the end of the 1-hour period.
- `dewpoint` - The dew point temperature at 2 meters above ground in degrees Celsius. Estimated for the end of the 1-hour period.
- `cloudcover_[low/mid/high/total]` - The percentage of the sky covered by clouds in the following altitude bands: 0-2 km, 2-6, 6+, and total. Estimated for the end of the 1-hour period.
- `10_metre_[u/v]_wind_component` - The [eastward/northward] component of wind speed measured 10 meters above surface in meters per second. Estimated for the end of the 1-hour period.
- `data_block_id`
- `forecast_datetime` - The timestamp of the predicted weather. Generated from `origin_datetime` plus `hours_ahead`. This represents the start of the 1-hour period for which weather data are forecasted.
- `direct_solar_radiation` - The direct solar radiation reaching the surface on a plane perpendicular to the direction of the Sun accumulated during the hour, in watt-hours per square meter.
- `surface_solar_radiation_downwards` - The solar radiation, both direct and diffuse, that reaches a horizontal plane at the surface of the Earth, accumulated during the hour, in watt-hours per square meter.
- `snowfall` - Snowfall over hour in units of meters of water equivalent.
- `total_precipitation` - The accumulated liquid, comprising rain and snow that falls on Earth's surface over the described hour, in units of meters.

### historical_weather.csv
[Historic weather data](https://open-meteo.com/en/docs).
- `datetime` - This represents the start of the 1-hour period for which weather data are measured.
- `temperature` - Measured at the end of the 1-hour period.
- `dewpoint` - Measured at the end of the 1-hour period.
- `rain` - Different from the forecast conventions. The rain from large scale weather systems of the hour in millimeters.
- `snowfall` - Different from the forecast conventions. Snowfall over the hour in centimeters.
- `surface_pressure` - The air pressure at surface in hectopascals.
- `cloudcover_[low/mid/high/total]` - Different from the forecast conventions. Cloud cover at 0-3 km, 3-8, 8+, and total.
- `windspeed_10m` - Different from the forecast conventions. The wind speed at 10 meters above ground in meters per second.
- `winddirection_10m` - Different from the forecast conventions. The wind direction at 10 meters above ground in degrees.
- `shortwave_radiation` - Different from the forecast conventions. The global horizontal irradiation in watt-hours per square meter.
- `direct_solar_radiation`
- `diffuse_radiation` - Different from the forecast conventions. The diffuse solar irradiation in watt-hours per square meter.
- `[latitude/longitude]` - The coordinates of the weather station.
- `data_block_id`

# 2. Setup

## Import

In [ ]:
import json
from datetime import date, time

import colorcet as cc
import holidays
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.transforms as mt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from IPython.display import Image
from matplotlib.dates import DateFormatter, MonthLocator
from matplotlib.lines import Line2D
from matplotlib.ticker import MultipleLocator
from pandas.core.groupby.generic import DataFrameGroupBy
from prophet import Prophet
from scipy.ndimage import binary_dilation
from shapely.geometry import shape
from statsmodels.tsa.seasonal import MSTL, seasonal_decompose

In [ ]:
RAND = 10
RAW_DATA = "../raw_data/"

train_df = pd.read_csv(f"{RAW_DATA}train.csv")
gas_prices_df = pd.read_csv(f"{RAW_DATA}gas_prices.csv")
client_df = pd.read_csv(f"{RAW_DATA}client.csv")
electricity_prices_df = pd.read_csv(f"{RAW_DATA}electricity_prices.csv")
forecast_weather_df = pd.read_csv(f"{RAW_DATA}forecast_weather.csv")
historical_weather_df = pd.read_csv(f"{RAW_DATA}historical_weather.csv")
station_county_mapping = pd.read_csv(
    f"{RAW_DATA}weather_station_to_county_mapping.csv"
)
county_id_to_name_map = pd.read_json(
    f"{RAW_DATA}county_id_to_name_map.json",
    typ="series",
).str.lower()

# External file with Estonian counties boundaries for visualisation
with open("../additional_data/estonia.geojson", "r", encoding="utf-8") as f:
    estonia_geojson = json.load(f)

In [ ]:
CATEGORICAL_DICT = {
    "county": county_id_to_name_map,
    "is_business": {0: "not_business", 1: "business"},
    "is_consumption": {0: "production", 1: "consumption"},
    "product_type": {
        0: "combined",
        1: "fixed",
        2: "general_service",
        3: "spot",
    },
}

PALETTE = sns.color_palette(
    cc.glasbey[:4]
    + [cc.glasbey[8]]
    + cc.glasbey[5:8]
    + [cc.glasbey[4]]
    + [cc.glasbey[12]]
    + cc.glasbey[10:12]
    + [cc.glasbey[9]]
    + cc.glasbey[13:16]
).as_hex()

SEGMENT_C = ["county", "product_type", "is_business"]
CATEGORICAL_C = ["county", "product_type", "is_business", "is_consumption"]
TARGET_C = [
    "county",
    "product_type",
    "is_business",
    "is_consumption",
    "datetime",
]

## Display Options

In [ ]:
pd.set_option(
    "display.float_format",
    lambda x: f"{x:.2e}" if abs(x) < 0.01 and x != 0 else f"{x:.2f}",
)
pd.set_option('display.max_columns', None)

In [ ]:
sns.set_style("whitegrid")

## Functions

In [ ]:
def print_right_filled_header(text: str) -> None:
    """
    Prints the provided header text followed by ">" characters, filling
    the line up to 100 characters.
    """
    total_length = 100
    text = text.strip()
    print("\n" + text + " " + ">" * (total_length - len(text)))

In [ ]:
def total_size(df: pd.DataFrame) -> np.int64:
    "Return total size of the dataframe in megabytes."
    return df.memory_usage().sum() // (1024**2)

In [ ]:
def columns_summary(df: pd.DataFrame) -> pd.DataFrame:
    """
    Summarize a dataframe with combined information from df.info() and
    df.describe(). Additionally, calculate the maximum length of the
    decimal part for numpy.float64 and numpy.float32 columns.

    Parameters
    ----------
    df : pd.DataFrame
        The input dataframe to summarize.

    Returns
    -------
    pd.DataFrame
        A combined summary dataframe.
    """

    column_info = {
        "column": df.columns,
        "not_na_count": [],
        "dtype": df.dtypes.values,
        "size_mb": (df.memory_usage().div(1024**2).round(2))[1:],
        "max_dec_len": [],
    }

    for column in df.columns:
        col_data = df[column]
        column_info["not_na_count"].append(col_data.notna().sum())

        if (not isinstance(col_data.dtype, pd.CategoricalDtype)) and (
            np.issubdtype(col_data.dtype, np.float64)
            or np.issubdtype(col_data.dtype, np.float32)
        ):
            decimal_len = (
                col_data.dropna()
                .apply(lambda x: len(str(x).split(".")[1]))
                .max()
            )
            column_info["max_dec_len"].append(decimal_len)
        else:
            column_info["max_dec_len"].append(-1)

    df = pd.merge(
        pd.DataFrame(
            data=column_info,
        ),
        df.describe(include="all").drop("count").T.reset_index(names="column"),
        on=["column"],
    )

    first_columns = [
        "column",
        "dtype",
        "size_mb",
        "max_dec_len",
        "min",
        "max",
    ]
    other_columns = [
        column for column in df.columns if column not in first_columns
    ]

    return df.loc[:, first_columns + other_columns]

In [ ]:
def df_show_info(df: pd.DataFrame) -> None:
    """
    Print the number of dataframe NaNs, the count of duplicated rows
    and display a summary of the dataframe.
    """

    print_right_filled_header("HEAD")
    display(df.head())

    print_right_filled_header("TAIL")
    display(df.tail())

    print_right_filled_header("MISSING VALUES")
    nan_counts = df.isna().sum()
    nan_series = nan_counts[nan_counts > 0]
    if nan_series.empty:
        print("No missing values.")
    else:
        display(nan_series)

    print_right_filled_header("FULL DUPLICATES")
    dup_counts = df.duplicated().sum()
    if dup_counts:
        display(f"Total duplicated rows: {dup_counts}")
    else:
        print("No duplicate rows.")

    print_right_filled_header("TOTAL SIZE")
    print(f"Total DataFrame size: {total_size(df)} MB.")

    print_right_filled_header("SUMMARY")
    display(columns_summary(df))

In [ ]:
def categorical_mapper(
    df: pd.DataFrame,
    values_mapper: dict,
) -> pd.DataFrame:
    """
    Map categorical columns in the dataframe to their corresponding
    values based on the provided mapper and convert these columns to
    categorical type.

    Parameters
    ----------
    df : pd.DataFrame
        Dataframe to process.
    values_mapper : dict
        A dictionary where keys are column names, and values are
        mapping dictionaries.

    Returns
    -------
    pd.DataFrame
        Modified dataframe with specified columns mapped to their
        categorical values.
    """
    for key, value in values_mapper.items():
        if key in df.columns:
            df[key] = df[key].map(value).astype("category")
        else:
            print(
                "categorical_mapper:",
                f"column '{key}' not found in dataframe, skipping."
            )

    return df

In [ ]:
def mismatched_indices_check(
    date_diff: pd.Series,
    id_diff: pd.Series,
    difference: np.float64,
) -> None:
    """
    Check if rows with an n-day difference in the date_diff have
    mismatched indices with id_diff rows that have a difference of n
    and print it.
    """

    print(
        f"Difference {difference}: ",[
            "no mismatches",
            "indices mismatch detected"][
        int(np.any(
            date_diff[date_diff == pd.Timedelta(difference, "day")]
            .index
            != id_diff[id_diff == difference].index))],
        ".",
        sep="",
    )

In [ ]:
def same_groups(dfgb: DataFrameGroupBy, column: str) -> pd.Series:
    """
    Combine values of the specified column in each subgroup into
    tuples, compare these tuples within each group, and return a
    pd.Series of boolean values indicating whether each tuple is a duplicate.
    """
    return dfgb[column].agg(tuple).duplicated()

In [ ]:
def circle_label(color: tuple | str, label: str) -> Line2D:
    """
    Create a handle for legend with a circular marker with specified
    color and label.
    """
    return Line2D(
        [],
        [],
        color=color,
        marker="o",
        linestyle="None",
        markersize=8,
        label=label,
    )

In [ ]:
def unique_combinations(df: pd.DataFrame, combination: list) -> pd.DataFrame:
    return (
        df[combination]
        .drop_duplicates()
        .sort_values(combination, ignore_index=True)
    )

# 3. Data Preparation

## Feature Commentary

From description:
- `datetime` - The Estonian time in EET (UTC+2) / EEST (UTC+3).
- `data_block_id` - All rows sharing the same `data_block_id` will be available at the same forecast time. This is a function of what information is available when forecasts are actually made, at 11 AM each morning. For example, if the forecast weather `data_block_id` for predictins made on October 31st is 100 then the historic weather `data_block_id` for October 31st will be 101 as the historic weather data is only actually available the next day.
- `row_id` - A unique identifier for the row.
- `prediction_unit_id` - A unique identifier for the `county`, `is_business`, and `product_type` combination. *New prediction units can appear or disappear in the test set*.
***
Also, the competition host <u>[provided](https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/discussion/455833)</u> this scheme along with comments:

Let’s say we are on day D at 11am. We want to predict next day D+1 net consumption from 00 to 23 for every hours.
<table style="border: 1px solid black; border-collapse: collapse; width: 100%;">
  <tr>
    <th style="border: 1px solid black;">Category</th>
    <th style="border: 1px solid black;">Weather forecast</th>
    <th style="border: 1px solid black;">Historical weather</th>
    <th style="border: 1px solid black;">Historical consumption and production / Client data</th>
    <th style="border: 1px solid black;">Electricity prices</th>
    <th style="border: 1px solid black;">Gas prices</th>
  </tr>
  <tr>
    <td style="border: 1px solid black;">Last available data</td>
    <td style="border: 1px solid black;">Forecast for every hours of D and D+1 (published on D)</td>
    <td style="border: 1px solid black;">Every hours until day D, 10 am</td>
    <td style="border: 1px solid black;">Every hours of Day D-1</td>
    <td style="border: 1px solid black;">Every hours of day D (published on D-1)</td>
    <td style="border: 1px solid black;">Data for day D (published on D-1)</td>
  </tr>
</table>

Prices are published everyday at 2 pm (so after 11 am), that is we do not have D+1 prices.
The data_block_id already reflects this timeline of availability of the data. There is no need to apply additional lag if joining on data_block_id.
***
Electricity and gas prices, dates, and `data_block_id` were additionally <u>[clarified](https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/discussion/456636#2533090)</u>:

> Q: Are the values in the gas & electricity data actual values or predicted?

> A: Actual values. Prices are published everyday for the next day. However we predict electricity consumption and production earlier, so next day's prices are not yet available when predicting next day's production and consumption
***
Later, I will explore whether there are any discrepancies in the correlations between `data_block_id` and `datetime`, or between `prediction_unit_id` and the `categorical features`. For now, I will analyze the data by:
1. Checking all dataframes using `describe()` and `info()`.
2. Verifying the presence of missing values or duplicates.

## train_df

In [ ]:
df_show_info(train_df)

- There are a few missing values (528/2,018,352 < 0.1%) and no duplicates in the dataframe. Only the `target` feature has missing values.
- The categorical features `county`, `is_business`, `product_type`, and `is_consumption` can be converted to the categorical data type. Their values can be renamed to improve clarity and understanding.
- The `target` feature can be converted to `float32` because its values range from 0.0 to 15,480.27, and the standard deviation (910.20) is relatively large compared to the mean (274.86), indicating a wide distribution of values. Given the dataset size (2,017,824 records), the precision of the fractional part becomes less significant, and `float32` provides adequate precision while reducing memory usage.
- The `datetime` feature can be converted to `datetime64[ns]` for proper handling of data.
- The `data_block_id` feature can be converted to `uint16` (range: 0 through 65,535) as its maximum value is small.
- The `row_id` feature is similar to the index values in the current default sorting. For now, it can be converted to `uint32` (range: 0 through 4,294,967,295) as its range fits well within this data type, and the use of `int64` is unnecessary because it supports negative numbers, which are irrelevant in this case.
- The `prediction_unit_id` feature can be converted to `uint8` (range: 0 through 255). Although new combinations may appear in the future, this data type is sufficient to uniquely represent all future combinations of current county, is_business, and product_type (16 * 2 * 4 = 96).

### Data Transformation

In [ ]:
# Reorder columns
train_df = train_df[
    [
        "county",
        "product_type",
        "is_business",
        "is_consumption",
        "datetime",
        "target",
        "data_block_id",
        "row_id",
        "prediction_unit_id",
    ]
]

# Rename categorical values to avoid confusion and improve readability
train_df = categorical_mapper(train_df, CATEGORICAL_DICT)

# Change data types to reduce memory usage
train_df = train_df.astype(
    {
        "target": "float32",
        "data_block_id": "uint16",
        "row_id": "uint32",
        "prediction_unit_id": "uint8",
        "datetime": "datetime64[ns]",
    }
)

print(f"Size after transformation: {total_size(train_df)} MB.")

### Partial Duplicates

In [ ]:
# Check timestamp and categorical features for duplicates
train_df.duplicated(TARGET_C).sum()

No partial duplicates.

### Missing Values

In [ ]:
# Display rows with missing values from the dataframe
train_df[train_df.isna().any(axis=1)].head()

The `datetime` values for rows with missing target values start from '2021-10-31 03:00:00'.

In [ ]:
# Create array with timestamps corresponding to rows with missing
# target values
na_datetimes = train_df[train_df.isna().any(axis=1)]["datetime"].unique()
na_datetimes

All missing values correspond to the start or end of daylight saving time.

In [ ]:
# Check that all values for these timestamps are missing
train_df.isna().values.sum() == train_df.loc[
    train_df["datetime"].isin(na_datetimes), ["target"]
].shape[0]

In [ ]:
# Check entries near NaNs
for timestamp in na_datetimes:
    display(
        train_df.loc[
            (
                # Loc on categories from the first row for simplicity
                (
                    train_df[CATEGORICAL_C] == train_df.iloc[0][CATEGORICAL_C]
                ).all(axis=1)
            )
            & (
                train_df["datetime"].isin(
                    pd.date_range(
                        timestamp - pd.Timedelta("1 h"),
                        timestamp + pd.Timedelta("1 h"),
                        freq="h",
                    )
                )
            ),
            ["datetime", "target"],
        ]
    )

The data for the switch from DST is filled similarly to the switch to DST; however, these are two distinct processes with significant differences. The spring switch adds one hour, which results in a missing row with `NaN`. The autumn switch subtracts one hour, which would typically create a duplicated timestamp. However, in the provided data, no duplicate timestamps are present; instead, a row with `NaN` appears again.

For example, the timestamp from 3-1 = 2 is duplicated (and is missing in the DataFrame), 4-1 = 3 results in a row with `NaN`, and only from 5-1 = 4 does the value appear as expected (not `NaN`).
Because of this, it is acceptable to use interpolation to impute the missing value for the autumn switch, while simply dropping the missing value for the spring switch. Other methods may also be applicable, particularly if it is important to preserve the one-hour structure of the time series, as will be discussed below.

### Consistency and Relationship

#### Row Identifier

In [ ]:
# Check if row_id values are equal to index values
(train_df["row_id"] != train_df.index).sum()

All `row_id` values are equal to index values.

In [ ]:
train_df = train_df.drop(columns=["row_id"])

#### Datetime and data_block_id

For predictions made for day D + 1, all historical consumption and production data should have `data_block_id` values equal to D - 1. This is because the `data_block_id` represents the data available at a specific time. Data for day D is unavailable at the time of prediction, as it corresponds to the current day, and no historical data exists for it yet.

To validate this, the `datetime` and `data_block_id` columns require a check to ensure that, in chronological order, both date and ID values progress uniformly. If a single date corresponds to multiple `data_block_id` values, it will be evident because the differences between consecutive rows in these two columns will not be consistent.

In [ ]:
# Create variables to store the differences between the current and
# previous row for all rows in datetime and data_block_id
train_date_diff = train_df["datetime"].dt.date.diff()
train_id_diff = train_df["data_block_id"].diff()

# Print unique values to verify that default order reflects sorting
# from oldest to newest without skips
print(train_date_diff.value_counts(dropna=False))
print(train_id_diff.value_counts(dropna=False))

- `NaT` and `NaN` values correspond to the first row in both columns, where no previous row data is available for comparison.
- A value of '0' indicates that the current and previous rows belong to the same day.
- A value of '1' indicates the transition from one day to the next.

In [ ]:
for difference in train_id_diff.unique():
    mismatched_indices_check(train_date_diff, train_id_diff, difference)

The default order of the raw data for this dataframe is sorted by datetime, and the `data_block_id` reflects the actual data availability, with no errors observed in the `datetime` column.

#### Time Series Gaps

In [ ]:
df_for_missing = train_df[TARGET_C + ["target"]].dropna().copy()

# Add hour index starting from the beginning
df_for_missing["hour_index"] = (
    (df_for_missing["datetime"] - df_for_missing["datetime"].min())
    // pd.Timedelta(hours=1)
).astype(np.uint16)

# Add a group index corresponding to the unique combination of all
# categorical features (i.e. this feature is not equal to
# prediction_unit_id): county, is_business, product_type,
# is_consumption.
# The maximum number of combinations is 16 * 4 * 2 * 2 = 256, but the
# actual number of observed values is less than 256
df_for_missing["group_index"] = (
    df_for_missing.groupby(CATEGORICAL_C, observed=True,)
    .ngroup()
    .astype(np.uint8)
)

# Create a 2D array with a shape equal to the number of groups and the
# total number of hours between the minimum and maximum timestamps
n_groups = df_for_missing["group_index"].nunique()
n_hours = df_for_missing["hour_index"].max() + 1
missmap = np.full((n_groups, n_hours), np.nan)

# Fill the array with the corresponding flag values
missmap[df_for_missing["group_index"], df_for_missing["hour_index"]] = (
    df_for_missing["target"] != 0
)

In [ ]:
# Dilate missing values because of high density of plot and isolated
# missing values are not visible.
n = 6 # How many hours before and after count as missing
structure = np.ones((1, 2 * n + 1), dtype=bool)
missing_dilated = binary_dilation(np.isnan(missmap), structure=structure)
missmap[missing_dilated] = np.nan

In [ ]:
alpha = 0.8

min_date = df_for_missing["datetime"].min()
max_date = df_for_missing["datetime"].max()
n_months = len(pd.date_range(start=min_date, end=max_date, freq="ME"))

fig, ax = plt.subplots(figsize=(14, 7))
sns.heatmap(
    missmap,
    cmap=sns.color_palette(["tab:blue", "tab:orange"]),
    cbar=False,
    alpha=alpha,
    ax=ax,
)

xtick_indices = np.linspace(0, n_hours, num=n_months, dtype=int)
xtick_labels = pd.date_range(min_date, max_date, freq="MS")

ax.set_xticks(xtick_indices)
ax.set_xticklabels(xtick_labels.strftime("%Y-%m"), rotation=45, ha="right")

dx, dy = 0, -2.5
for label in ax.get_yticklabels():
    offset = mt.ScaledTranslation(dx / 72, dy / 72, fig.dpi_scale_trans)
    label.set_transform(label.get_transform() + offset)

legend_patches = [
    mpatches.Patch(
        facecolor="white",
        label="Missing value",
        edgecolor="black",
        linewidth=0.5,
    ),
    mpatches.Patch(
        facecolor="tab:blue",
        alpha=alpha,
        label="Zero value",
        edgecolor="black",
        linewidth=0.5,
    ),
    mpatches.Patch(
        facecolor="tab:orange",
        alpha=alpha,
        label="Not zero value",
        edgecolor="black",
        linewidth=0.5,
    ),
]

plt.legend(
    handles=legend_patches,
    title="Data Presence",
    title_fontsize=12,
    bbox_to_anchor=(1, 1),
    loc="upper left",
    fontsize=11,
    frameon=False,
)

plt.title(
    "Heatmap of time series gaps for all combinations "
    "of categorical features",
    fontsize=13,
)
plt.xlabel("Month", fontsize=10)
plt.ylabel("Group index", fontsize=10)
plt.grid(False)
plt.show()

The plot shows that time series across different groups have varying start and end dates. Some groups have large gaps in their data, and the overall data volume varies between groups. Noticeably more zero values occur in winter than in summer, and these zeros appear every other hour - suggesting electricity production patterns. Missing values due to the switch to or from DST ([1 hour shift](#Missing-Values)) are displayed as vertical white lines.

#### Categorical Features Ratio

In [ ]:
# Check that each combination of county, product_type, and
# is_business corresponds to exactly one unique prediction_unit_id
(
    train_df.groupby(
        SEGMENT_C,
        observed=True,
    )["prediction_unit_id"].nunique()
    != 1
).sum()

All combinations of `county`, `is_business`, and `product_type` correspond to a single `prediction_unit_id`, with no errors.

In [ ]:
for c in CATEGORICAL_C:
    print(train_df[c].value_counts())
    print()

Classes in each category except `is_consumption` are imbalanced.

In [ ]:
# Check that in each segment the number of production and consumptuon
# values are the same
train_df.groupby(CATEGORICAL_C, as_index=False, observed=True)[
    "target"
].count().groupby(SEGMENT_C, as_index=False, observed=True)[
    "target"
].nunique()[
    "target"
].nunique()

Yes, in each segment (combination of `county`, `product_type`, `is_business`) the count of `target` entries is identical for `production` and `consumption`.

In [ ]:
# Plot the visualisation of the number of entries for each unique
# combination of categorical features

# Create a DF with all segments and their counts of target values
train_categories = train_df.groupby(
    SEGMENT_C,
    as_index=False,
    observed=True,
)["target"].count()

# Create columns with concatenated labels for segments
train_categories["group"] = (
    train_categories[SEGMENT_C].astype(str).agg(" ".join, axis=1)
)

fig, ax = plt.subplots(figsize=(15, 3))
sns.barplot(
    data=train_categories,
    x="group",
    y="target",
    hue="county",
    palette=PALETTE,
    width=1,
    linewidth=0.4,
)

handles, labels = ax.get_legend_handles_labels()
labels = [label.capitalize() for label in labels]
plt.legend(
    handles=handles,
    labels=labels,
    title="County",
    title_fontsize=12,
    bbox_to_anchor=(1.015, 1.1),
    loc="upper left",
    borderaxespad=0,
    markerscale=3,
    frameon=False,
    fontsize=11,
)

plt.title("Number of records for each segment", fontsize=13)
plt.grid(False)

ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(
    train_categories["group"].str.split(n=1).str[1],
    rotation=90,
    fontsize=10,
)
plt.xlabel("Group", fontsize=10)
plt.show()

The number of entries in each segment ranges from around 4,000 to 30,000, and the number of subgroups also varies across counties.

## gas_prices_df

In [ ]:
df_show_info(gas_prices_df)

- There are no missing values or duplicates in the dataframe.
- The `data_block_id` values start from 1, unlike the `data_block_id` values in train_df, which start from 0. This difference exists because </u>[`[lowest/highest]_price_per_mwh` are end-of-day prices and aren't available in the late morning when forecasts are made](https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/discussion/453355#2515054)</u>.

### Data Transformation

In [ ]:
# Reorder columns
gas_prices_df = gas_prices_df[
    [
        "origin_date",
        "forecast_date",
        "data_block_id",
        "lowest_price_per_mwh",
        "highest_price_per_mwh",
    ]
]

# Change data types to reduce memory usage
gas_prices_df = gas_prices_df.astype(
    {
        "origin_date": "datetime64[ns]",
        "forecast_date": "datetime64[ns]",
        "data_block_id": "uint16",
        "lowest_price_per_mwh": "float32",
        "highest_price_per_mwh": "float32",
    }
)

### Partial Duplicates

In [ ]:
# Check timestamps for duplicates
gas_prices_df.duplicated(["forecast_date", "origin_date"]).sum()

No partial duplicates.

### Consistency and Relationship Validation
#### Origin and forecast dates
Due to the strong correlation between the two datetime columns (at least based on the description and a few observed rows), it's necessary to verify if this correlation holds true.

In [ ]:
# Check that each origin date is the next day compared to the previous
# one
gas_date_diff = gas_prices_df["origin_date"].diff()
print(gas_date_diff.value_counts(dropna=False))

- Yes, all values are chronological days without skips. The `NaT` value corresponds to the first date, where no previous date exists for comparison.

In [ ]:
# Check if any forecast dates are not exactly one day after origin dates
(
    gas_prices_df["origin_date"] + pd.Timedelta(days=1)
    != gas_prices_df["forecast_date"]
).sum()

- All `origin_date` values are correct and represent the day before their corresponding `forecast_date` values. This means that correlation between these features is 1, and one of them can likely be deleted.
- The raw data in this dataframe is sorted chronologically by datetime.

#### Origin date and data_block_id

In [ ]:
gas_id_diff = gas_prices_df["data_block_id"].diff()
print(gas_id_diff.value_counts(dropna=False))

In [ ]:
for difference in gas_id_diff.unique():
    mismatched_indices_check(gas_date_diff, gas_id_diff, difference)

- Similar to `origin_date`, the first value is missing, and all subsequent values have a difference of 1. Since the differences are consistent across both columns, it can be concluded that there are no errors between `origin_date` and `data_block_id`.

## client_df

In [ ]:
df_show_info(client_df)

- There are no missing values or duplicates in the dataframe.
- The `data_block_id` values start from 2 instead of 0 or 1, as mentioned earlier.

### Data Transformation

In [ ]:
# Reorder columns
client_df = client_df[
    [
        "county",
        "product_type",
        "is_business",
        "date",
        "eic_count",
        "installed_capacity",
        "data_block_id",
    ]
]

# Rename categorical values to avoid confusion and improve readability
client_df = categorical_mapper(client_df, CATEGORICAL_DICT)

# Change data types to reduce memory usage
client_df = client_df.astype(
    {
        "date": "datetime64[ns]",
        "eic_count": "uint32",
        "installed_capacity": "float32",
        "data_block_id": "uint16",
    }
)

### Partial duplicates

In [ ]:
# Check timestamp and categorical features for duplicates
client_df.duplicated(["county", "product_type", "is_business", "date"]).sum()

No partial duplicates.

### Consistency and Relationship Validation
#### Date and data_block_id
As with the previous dataframes, it is necessary to validate the correlation between `date` and `data_block_id`.

In [ ]:
# Create variables to store the differences between the current and
# previous row for all rows in date and data_block_id
client_date_diff = client_df["date"].dt.date.diff()
client_id_diff = client_df["data_block_id"].diff()


# Print unique values to verify that default order reflects sorting
# from oldest to newest without skips
print(client_date_diff.value_counts(dropna=False))
print(client_id_diff.value_counts(dropna=False))

- `NaT` and `NaN` values correspond to the first row in both columns, where no previous row data is available for comparison.
- A value of '0' indicates that the current and previous rows belong to the same day.
- A value of '1' indicates the transition from one day to the next.

In [ ]:
for difference in client_id_diff.unique():
    mismatched_indices_check(client_date_diff, client_id_diff, difference)

- The default order of the raw data for this dataframe is sorted by datetime, and the `data_block_id` reflects the actual data availability, with no errors observed in the `date` column.

## electricity_prices_df

In [ ]:
df_show_info(electricity_prices_df)

- There are no missing values or duplicates in the dataframe.
- The `data_block_id` values start from 1.
- The minimum value of the `euros_per_mwh` column is negative.

### Data Transformation

In [ ]:
# Change data types to reduce memory usage
electricity_prices_df = electricity_prices_df.astype(
    {
        "origin_date": "datetime64[ns]",
        "forecast_date": "datetime64[ns]",
        "euros_per_mwh": "float32",
        "data_block_id": "uint16",
    }
)

# Datetime at which the forecast will be available for the model
electricity_prices_df["electricity_datetime"] = electricity_prices_df[
    "origin_date"
] + pd.Timedelta(2, "d")

# Reorder columns
electricity_prices_df = electricity_prices_df[
    [
        "electricity_datetime",
        "euros_per_mwh",
        "data_block_id",
        "origin_date",
        "forecast_date",
    ]
]

# Rename to reflect hourly timestamps instead of just dates
electricity_prices_df = electricity_prices_df.rename(
    columns={
        "origin_date": "origin_datetime",
        "forecast_date": "forecast_datetime",
    }
)

### Partial Duplicates

In [ ]:
# Check timestamps for duplicates
electricity_prices_df.duplicated(["origin_datetime", "forecast_datetime"]).sum()

No partial duplicates.

### Non-positive Prices

In [ ]:
electricity_prices_df[electricity_prices_df["euros_per_mwh"] <= 0].shape

According to the [<u>official comment</u>](https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/discussion/454932#2523730), such prices are not an error.

[<u>Negative pricing</u>](https://en.wikipedia.org/wiki/Negative_pricing) can occur when demand for a product drops or supply increases to an extent that owners or suppliers are prepared to pay others to accept it, in effect setting the price to a negative number.

### Consistency and Relationship Validation
As with `gas_prices_df`, it is necessary to validate the correlation between `origin_datetime`, `forecast_datetime`, and `data_block_id`.
#### Origin and forecast dates

In [ ]:
# Check that each electricity_origin_date is the next hour compared to
# the previous one
print(
    electricity_prices_df["origin_datetime"]
    .diff()
    .value_counts(dropna=False)
)

There are also values with a 2-hour difference. This might indicate an issue with data collection or other inconsistencies.

In [ ]:
electricity_prices_df[
    electricity_prices_df["origin_datetime"].diff()
    == pd.Timedelta(2, "hour")
]

A 2-hour time difference occurs at 03:00 on `2022-03-27` and `2023-03-26` due to the transition to daylight saving time. This transition results in an increased time difference between records.

In [ ]:
# Check if any forecast dates are not exactly one day after origin dates
(
    electricity_prices_df["origin_datetime"]
    + pd.Timedelta(days=1)
    != electricity_prices_df["forecast_datetime"]
).sum()

- All `origin_datetime` values represent the day before their corresponding `forecast_datetime` values.
- The raw data in this dataframe is sorted chronologically by datetime.

#### Origin date and data_block_id

In [ ]:
# Create variables to store the differences between the current and
# previous row for all rows in origin_datetime and data_block_id
electricity_date_diff = electricity_prices_df[
    "origin_datetime"
].dt.date.diff()
electricity_id_diff = electricity_prices_df["data_block_id"].diff()

# Print unique values to verify that default order reflects sorting
# from oldest to newest without skips
print(electricity_date_diff.value_counts(dropna=False))
print(electricity_id_diff.value_counts(dropna=False))

- `NaT` and `NaN` values correspond to the first row in both columns, where no previous row data is available for comparison.
- A value of '0' indicates that the current and previous rows belong to the same day.
- A value of '1' indicates the transition from one day to the next.

In [ ]:
for difference in electricity_id_diff.unique():
    mismatched_indices_check(
        electricity_date_diff, electricity_id_diff, difference
    )

- The default order of the raw data for this dataframe is sorted by datetime, and the `data_block_id` reflects the actual data availability, with no errors observed in the `origin_datetime` column.

## forecast_weather_df

In [ ]:
df_show_info(forecast_weather_df)

- Weather forecasts are generated at 2 AM for the next 48 hours. Since the goal is to predict consumption and production for every hour of the following day, only forecasts between +22 and +45 hours_ahead will be used. However, other predictions from the same day can also be utilized as additional predictors.
- There are 2 missing values and zero duplicates in the dataframe. Only the `surface_solar_radiation_downwards` column contains missing values. 
- The `data_block_id` values start from 1.

### Data Transformation

In [ ]:
# Reorder columns
forecast_weather_df = forecast_weather_df[
    [
        "latitude",
        "longitude",
        "origin_datetime",
        "hours_ahead",
        "forecast_datetime",
        "data_block_id",
        "temperature",
        "dewpoint",
        "cloudcover_low",
        "cloudcover_mid",
        "cloudcover_high",
        "cloudcover_total",
        "10_metre_u_wind_component",
        "10_metre_v_wind_component",
        "direct_solar_radiation",
        "surface_solar_radiation_downwards",
        "snowfall",
        "total_precipitation",
    ]
]

# Change data types to reduce memory usage
forecast_weather_df[["latitude", "longitude"]] = (
    forecast_weather_df[["latitude", "longitude"]].round(1).mul(10)
)
forecast_weather_df[
    ["cloudcover_low", "cloudcover_mid", "cloudcover_high", "cloudcover_total"]
] = (
    forecast_weather_df[
        [
            "cloudcover_low",
            "cloudcover_mid",
            "cloudcover_high",
            "cloudcover_total",
        ]
    ]
    .round(2)
    .mul(100)
)

forecast_weather_df = forecast_weather_df.astype(
    {
        "latitude": "uint16",
        "longitude": "uint16",
        "origin_datetime": "datetime64[ns]",
        "forecast_datetime": "datetime64[ns]",
        "data_block_id": "uint16",
        "temperature": "float32",
        "dewpoint": "float32",
        "cloudcover_low": "uint8",
        "cloudcover_mid": "uint8",
        "cloudcover_high": "uint8",
        "cloudcover_total": "uint8",
        "10_metre_u_wind_component": "float32",
        "10_metre_v_wind_component": "float32",
        "direct_solar_radiation": "float32",
        "surface_solar_radiation_downwards": "float32",
        "snowfall": "float32",
        "total_precipitation": "float32",
    }
)
forecast_weather_df["hours_ahead"] = pd.to_timedelta(
    forecast_weather_df["hours_ahead"], "h"
)

print(f"Size after transformation: {total_size(forecast_weather_df)} MB.")

### Partial Duplicates

In [ ]:
# Check coordinates and timestamps for duplicates
forecast_weather_df.duplicated(
    [
        "latitude",
        "longitude",
        "origin_datetime",
        "hours_ahead",
        "forecast_datetime",
    ]
).sum()

No partial duplicates.

### Missing Values

In [ ]:
forecast_weather_df[forecast_weather_df.isna().any(axis=1)]

- Since there are only two missing values, it could be due to a local issue lasting for just 2 hours.

### Consistency and Relationship Validation
Ensure `origin_datetime`, `forecast_datetime`, `hours_ahead` values are coherent within respective `latitude` and `longitude` groups.
#### Origin datetime

In [ ]:
# Sameness check
forecast_geo_gb = forecast_weather_df.groupby(["latitude", "longitude"])
same_groups(forecast_geo_gb, "origin_datetime").value_counts()

This dataframe contains 112 unique combinations of `latitude` and `longitude`, representing distinct geographical points for weather forecasts. Only one `False` value corresponds to the first group, indicating that all other location points have identical subsequences of `origin_datetime`.

In [ ]:
# Forecast creation times
forecast_weather_df["origin_datetime"].dt.time.value_counts()

All forecasts are made at either 1 AM or 2 AM, likely due to shifts associated with transitions to and from daylight saving time.

In [ ]:
# Select latitude and longitude values for the first entry of the
# geographical group, as the sequences of origin_datetime values are
# identical across each group
lat0, lon0 = forecast_weather_df.iloc[0][["latitude", "longitude"]]
fw_subset_df = forecast_weather_df[
    (forecast_weather_df["latitude"] == lat0)
    & (forecast_weather_df["longitude"] == lon0)
]

# Calculate the differences between consecutive origin_datetime values
# for the selected geographical group
fod_diff = fw_subset_df["origin_datetime"].diff()

# Display the counts of unique differences between consecutive
# origin_datetime entries
fod_diff.value_counts(dropna=False)

- `0 days 00:00:00`: rows corresponding to the same location and day, but with different `hours_ahead` values.
- `1 days 00:00:00`: indicates a shift from previous origin day.
- `0 days 23:00:00`: likely caused by a transition to daylight saving time.
- `1 days 01:00:00`: likely caused by a transition from daylight saving time.
- `NaT`: corresponds to the first rows in each group.

Based on these differences, it can be concluded that the data in this dataframe is sorted by `origin_datetime` by default.

In [ ]:
# Select rows where differences in origin_datetime are not 0, 1 day, or
# NaT
unusual_diff_df = fw_subset_df[
    (fod_diff != pd.Timedelta(0))
    & (fod_diff != pd.Timedelta(1, "d"))
    & (~fod_diff.isna())
][["origin_datetime"]]

# Merge the filtered timestamps with their corresponding time
# differences
pd.merge(
    unusual_diff_df,
    fod_diff[unusual_diff_df.index].rename("difference"),
    left_index=True,
    right_index=True,
)

Yes, the autumn shift corresponds to a switch to winter time, and the spring shift corresponds to a switch to summer time. Specifically:
- The `origin_datetime` hour is `02:00:00` during the following periods:
  - From `2021-09-01` to `2021-10-31`
  - From `2022-03-28` to `2022-10-30`
  - From `2023-03-27` to `2023-05-30`
- The `origin_datetime` hour is `01:00:00` during the following periods:
  - From `2021-11-01` to `2022-03-27`
  - From `2022-10-31` to `2023-03-26`

#### Forecast datetime
From the data description:
- `forecast_datetime` - The timestamp of the predicted weather. Generated from `origin_datetime` plus `hours_ahead`. This represents the start of the 1-hour period for which weather data are forecasted.

In [ ]:
# Sameness check for other two datetime features
for column in ("forecast_datetime", "hours_ahead"):
    print(same_groups(forecast_geo_gb, column).value_counts())

As with `origin_datetime`, for each geographical group, the sequence of `forecast_datetime` or `hours_ahead` values is identical across the locations. Therefore, it is sufficient to check the `forecast_datetime` and `hours_ahead` features using data from just one location.

In [ ]:
# Verify the equality stated in the dataset description:
# origin_datetime + hours_ahead = forecast_datetime
unusual_sum_df = fw_subset_df[
    fw_subset_df["origin_datetime"]
    + fw_subset_df["hours_ahead"]
    != fw_subset_df["forecast_datetime"]
]
unusual_sum_df["origin_datetime"].unique()

Values that do not satisfy the condition occur only within the 48 hours preceding a switch to or from DST.

In [ ]:
# Check that entries not satisfying the condition will satisfy it after
# adding or subtracting respectively.
# For autumn,
# origin_datetime + hours_ahead == forecast_datetime + 1 hour.
# For spring,
# origin_datetime + hours_ahead == forecast_datetime - 1 hour.

(unusual_sum_df["origin_datetime"]
 + unusual_sum_df["hours_ahead"]
 != unusual_sum_df["forecast_datetime"]
 + pd.to_timedelta(np.where(
     unusual_sum_df["origin_datetime"].dt.month == 10,  # October
     1,  # Add 1 hour for autumn switch to DST (October)
     -1  # Subtract 1 hour for spring switch from DST (March)
 ), 'h')
).sum()

All conditions are satisfied after the corresponding addition or subtraction. This means that all previously unsatisfied sums of `origin_datetime` and `hours_ahead` differ from `forecast_datetime` by exactly 1 hour due to the switch to or from DST.

#### data_block_id

In [ ]:
# Create variables to store the differences between the current and
# previous row for all rows in datetime and data_block_id

fw_id_diff = forecast_weather_df["data_block_id"].diff()
fw_date_diff = forecast_weather_df["origin_datetime"].dt.date.diff()

print(train_id_diff.value_counts(dropna=False))

- A value of '0' indicates that the current and previous rows belong to the same day.
- A value of '1' indicates the transition from one day to the next.
- `NaN` values correspond to the first row in both columns, where no previous row data is available for comparison.

In [ ]:
for difference in fw_id_diff.unique():
    mismatched_indices_check(fw_date_diff, fw_id_diff, difference)

The default order of the raw data for this dataframe is sorted by datetime, and the `data_block_id` reflects the actual data availability, with no errors observed in the `origin_datetime` column.

## historical_weather_df

In [ ]:
df_show_info(historical_weather_df)

- The `data_block_id` column type is `float64`, unlike in other dataframes.

In [ ]:
# Checking if all values in data_block_id are 
# integers to avoid errors and allow conversion to int type

print(np.unique(historical_weather_df.data_block_id.unique() %1))

No anomalous values; all have a remainder of 0 and can be converted to `uint16` type (which can store values from 0 to 65535), as the minimum value is 1 and the maximum is 637.

- There are no missing values or duplicates in the dataframe.
- The `data_block_id` is `float64` because the data in the original file is in "n.0" format.
- The `data_block_id` values start from 1.
- The `cloudcover_...` values are integers from 0 to 100, which differ from those in forecast_weather_df.

### Data Transformation

In [ ]:
# Reorder columns
historical_weather_df = historical_weather_df[
    [
        "latitude",
        "longitude",
        "datetime",
        "data_block_id",
        "temperature",
        "dewpoint",
        "rain",
        "snowfall",
        "surface_pressure",
        "cloudcover_total",
        "cloudcover_low",
        "cloudcover_mid",
        "cloudcover_high",
        "windspeed_10m",
        "winddirection_10m",
        "shortwave_radiation",
        "direct_solar_radiation",
        "diffuse_radiation",
    ]
]

# Change data types to reduce memory usage
historical_weather_df[["latitude", "longitude"]] = (
    historical_weather_df[["latitude", "longitude"]].round(1).mul(10)
)
historical_weather_df = historical_weather_df.astype(
    {
        "latitude": "uint16",
        "longitude": "uint16",
        "datetime": "datetime64[ns]",
        "data_block_id": "uint16",
        "temperature": "float32",
        "dewpoint": "float32",
        "rain": "float32",
        "snowfall": "float32",
        "surface_pressure": "float32",
        "cloudcover_total": "uint8",
        "cloudcover_low": "uint8",
        "cloudcover_mid": "uint8",
        "cloudcover_high": "uint8",
        "windspeed_10m": "float32",
        "winddirection_10m": "uint16",
        "shortwave_radiation": "uint16",
        "direct_solar_radiation": "uint16",
        "diffuse_radiation": "uint16",
    }
)

# Shift the timestamp by 24 hours so that it reflects the previous
# day's weather aligning it with the current day for merging.
# historical_weather_df["datetime"] = historical_weather_df[
#     "datetime"
# ] + pd.Timedelta("24 h")

print(f"Size after transformation: {total_size(historical_weather_df)} MB.")

### Partial Duplicates

In [ ]:
# Check coordinates and timestamps for duplicates
historical_weather_df.duplicated(["latitude", "longitude", "datetime"]).sum()

In [ ]:
hw_dups = historical_weather_df[
    historical_weather_df.duplicated(
        ["latitude", "longitude", "datetime"], keep=False
    )
]
hw_dups

In both duplicate pairs, one entry contains more zeros than the other. Duplicates can be processed by selecting one entry from each pair or creating an entry with mean values. It is helpful to check other nearby entries.

In [ ]:
historical_weather_df.loc[
    (
        historical_weather_df["latitude"]
        == historical_weather_df.iloc[hw_dups.index[0]].latitude
    )
    & (
        historical_weather_df["longitude"]
        == historical_weather_df.iloc[hw_dups.index[0]].longitude
    )
    & (
        historical_weather_df["datetime"].isin(
            pd.date_range(
                start=historical_weather_df.iloc[hw_dups.index[0]].datetime
                - pd.Timedelta("2 h"),
                periods=5,
                freq="h",
            )
        )
    )
]

In this case, entry `1176339` looks less probable than entry `1176340`.

In [ ]:
hw_to_drop = [1176339]

In [ ]:
historical_weather_df.loc[
    (
        historical_weather_df["latitude"]
        == historical_weather_df.iloc[hw_dups.index[2]].latitude
    )
    & (
        historical_weather_df["longitude"]
        == historical_weather_df.iloc[hw_dups.index[2]].longitude
    )
    & (
        historical_weather_df["datetime"].isin(
            pd.date_range(
                start=historical_weather_df.iloc[hw_dups.index[2]].datetime
                - pd.Timedelta("2 h"),
                periods=5,
                freq="h",
            )
        )
    )
]

In this case, entry `1176343` looks less probable than entry `1176342`.

In [ ]:
hw_to_drop.append(1176343)

In [ ]:
historical_weather_df = historical_weather_df.drop(
    index=hw_to_drop
).reset_index(drop=True)

### Consistency and Relationship Validation
Ensure `datetime` values are coherent within respective `latitude` and `longitude` groups.
#### Datetime

In [ ]:
# Sameness check
historical_geo_gb = historical_weather_df.groupby(["latitude", "longitude"])
sg_hw = same_groups(historical_geo_gb, "datetime")
sg_hw.value_counts()

Unlike in `forecast_weather_df`, this dataframe contains 2 unique `datetime` sequences.
This dataframe contains 112 unique combinations of `latitude` and `longitude` (after dropping two entries), representing distinct geographical points for historical weather data. Only one `False` value corresponds to the first group, indicating that all other location points have identical subsequences of `datetime`.

In [ ]:
h_sample = historical_weather_df[
    (historical_weather_df["latitude"] == lat0)
    & (historical_weather_df["longitude"] == lon0)
]

In [ ]:
# Display the counts of unique differences between consecutive datetime
# values for one geographical pount.
h_sample["datetime"].diff().value_counts(dropna=False)

- `NaT`: corresponds to the first row.
- `0 days 01:00:00`: indicates the transition from one hour to the next.

Based on these differences, it can be concluded that the data in this dataframe is sorted by `datetime` by default.

#### data_block_id

In [ ]:
historical_weather_df['data_block_id'].diff().value_counts(dropna=False)

All `data_block_id` values also increase incrementally.

In [ ]:
# Check that all data_block_id entries increment at 11 AM as stated in
# the dataset description
h_sample.loc[
    h_sample["data_block_id"].diff() == 1, "datetime"
].dt.time.value_counts()

## county_id_to_name_map

In [ ]:
county_id_to_name_map

In [ ]:
county_id_to_name_map = county_id_to_name_map.str.lower()

The value `unknown` can be interpreted in multiple ways. Firstly, it may represent missing values. Secondly, it could indicate a business that spans a very large area and is physically located in multiple counties.

## station_county_mapping

In [ ]:
df_show_info(station_county_mapping)

- More than half of the values are missing in two columns. This is likely because the dataframe covers the outer areas around Estonia.
- There are 15 unique `county` values plus a `NaN` value, whereas the other two dataframes that contain county data have 16 unique values without any `NaN`. This is probably because `unknown` might not have been used as a placeholder for missing data in this dataframe.

In [ ]:
print(
    station_county_mapping["county"].drop_duplicates().sort_values().tolist()
)

Value `12` is not present in this list. To visualize all locations on the map, `NaN` values could be filled with `12`, since it might represent an `unknown` value from other dataframes. However, these locations might be outside Estonia and may not correspond to an `unknown` county in the other dataframes.

### Data Transformation

In [ ]:
# Reorder columns
station_county_mapping = station_county_mapping[
    [
        "latitude",
        "longitude",
        "county_name",
        "county",
    ]
]

station_county_mapping[["latitude", "longitude"]] = (
    station_county_mapping[["latitude", "longitude"]].round(1).mul(10)
)

station_county_mapping[["county_name", "county"]] = station_county_mapping[
    ["county_name", "county"]
].fillna({"county_name": "unknown", "county": 12})

station_county_mapping["county_name"] = station_county_mapping[
    "county_name"
].str.lower()

station_county_mapping = (
    station_county_mapping.astype(
        {
            "latitude": "uint16",
            "longitude": "uint16",
            "county_name": "category",
            "county": "uint8",
        }
    )
    .astype({"county": "category"})
    .sort_values(["latitude", "longitude"], ignore_index=True)
)

station_county_mapping = station_county_mapping.rename(
    columns={
        "county_name": "county",
        "county": "county_index",
    }
)

# 4. EDA

## Locations

Verify that all unique locations in `forecast_weather_df`, `historical_weather_df`, and `station_county_mapping` match.

In [ ]:
(
    unique_combinations(forecast_weather_df, ["latitude", "longitude"])
    != unique_combinations(historical_weather_df, ["latitude", "longitude"])
).sum()

In [ ]:
(
    unique_combinations(forecast_weather_df, ["latitude", "longitude"])
    != station_county_mapping[["latitude", "longitude"]]
).sum()

Coordinates of all points from the three dataframes are identical.


#### Visualize all unique location points on an Estonia map

In [ ]:
# Extract county names and their centroid coordinates for labeling
region_names, centroid_lats, centroid_lons = [], [], []

for feature in estonia_geojson["features"]:
    region_name = feature["properties"].get("name")
    lat = round(shape(feature["geometry"]).centroid.y, 2)
    lon = round(shape(feature["geometry"]).centroid.x, 2)

    region_names.append(region_name)
    centroid_lats.append(lat)
    centroid_lons.append(lon)

In [ ]:
fig = px.scatter_map(
    lat=station_county_mapping.sort_values("county_index")["latitude"] / 10,
    lon=station_county_mapping.sort_values("county_index")["longitude"] / 10,
    color=station_county_mapping.sort_values("county_index")["county"],
    zoom=6.5,
    width=1360,
    height=820,
    color_discrete_sequence=PALETTE,
)

fig.update_layout(
    showlegend=False,
    map=dict(
        layers=[
            dict(
                sourcetype="geojson",
                source=estonia_geojson,
                type="line",
                color="black",
                line=dict(width=0.5),
            )
        ]
    ),
)

fig.add_trace(
    go.Scattermap(
        lat=centroid_lats,
        lon=centroid_lons,
        mode="text",
        text=region_names,
        textfont=dict(size=12, color="black"),
    )
)
fig.update_traces(marker=dict(size=16))

# fig.show()

fig.write_image("map.png")
Image("map.png")

## DataFrames Merging

For the `unknown` county in the target, both forecast and historical weather data will be aggregated from all counties, but data from `station_county_mapping` with `12` (`NaN`) values for the county (corresponding to geopoints in water or outside Estonia's boundaries) will be excluded. While the option of adding border or water points for the counties can be considered, such combinations will not be used at the moment.

Additionally, an extra flag feature indicating daylight saving time is necessary, as it influences various factors, such as the start of the workday.

In [ ]:
def avg_weather_data(df: pd.DataFrame, mapper: pd.DataFrame) -> pd.DataFrame:
    """
    Compute mean aggregated weather data per county and overall.

    This function creates a deep copy of the input DataFrame, merges it
    with the county mapping, and computes the mean of all numerical
    weather features grouped by time-related columns and county, as
    well as the overall mean across all counties.

    Parameters
    ----------
    df : pd.DataFrame
        Input weather data with latitude, longitude, datetime,
        numerical weather features.
    mapper : pd.DataFrame
        DataFrame that maps each location to a county containing
        "county", "latitude", and "longitude" columns. Only rows with a
        known "county" (i.e., not 'unknown') are used for merging.

    Returns
    -------
    pd.DataFrame
        DataFrame with mean aggregated weather data per county and overall.
    """

    # Merge weather data with county mapping, excluding 'unknown'
    # values which correspond to locations outside of Estonia
    df = df.copy()
    df = pd.merge(
        left=mapper.loc[
            mapper["county"] != "unknown",
            ["county", "latitude", "longitude"],
        ],
        right=df,
        how="left",
        on=["latitude", "longitude"],
        validate="1:m",
    )

    # Identify and exclude grouping columns from average calculation
    groups = [
        c
        for c in df.columns.tolist()
        if "datetime" in c or c == "hours_ahead" or c == "data_block_id"
    ]
    excluded_c = groups + ["county", "latitude", "longitude",]
    avg_c = [c for c in df.columns if c not in excluded_c]
    dtypes = df[avg_c].dtypes.to_dict()
    to_round = [k for k, v in dtypes.items() if np.issubdtype(v, np.integer)]

    # Compute overall mean (county='unknown'), then per-county mean,
    # and concatenate both results
    df = pd.concat(
        [
            df.groupby(groups, as_index=False, observed=True)[avg_c]
            .mean()
            .assign(county="unknown"),
            df.groupby(groups + ["county"], as_index=False, observed=True)[
                avg_c
            ].mean(),
        ],
        ignore_index=True,
    )
    dtypes.update({"county": "category"})

    # Round integer-derived columns back to integers and cast all
    # columns to original dtypes
    df[to_round] = df[to_round].round()
    df = df.astype(dtypes)
    return df[["county"] + groups + avg_c]

In [ ]:
def get_lag(df: pd.DataFrame, dt: str, lag: int, c: str) -> pd.DataFrame:
    """
    Shift 'dt' column by 'lag' days and rename the 'c' column.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame.
    dt : str
        Name of the datetime column to be shifted.
    lag : int
        Number of days to shift.
    c : str
        Name of the column to rename.

    Returns
    -------
    pd.DataFrame
        DataFrame with the shifted datetime column and renamed target
        column.

    Raises
    ------
    ValueError
        If 'lag' is less than 2.
    """
    if lag < 2:
        raise ValueError(f"'lag' must be at least 2 days, got {lag}")
    return df.assign(**{dt: df[dt] + pd.Timedelta(days=lag)}).rename(
        columns={c: f"{lag}d_lag_{c}"}
    )

In [ ]:
def get_moving_average(
    dfgb: DataFrameGroupBy,
    c: str,
    window: int,
) -> pd.DataFrame:
    """
    Compute rolling mean for a specified column of a grouped DataFrame
    and shift the datetime column by 48 hours.

    Parameters
    ----------
    dfgb : DataFrameGroupBy
        Grouped DataFrame (result of df.groupby(..., as_index=False)),
        where the original DataFrame was sorted by the datetime index.
    c : str
        Name of the column to aggregate.
    window : int
        Rolling window size in hours (min_periods=window).

    Returns
    -------
    pd.DataFrame
        DataFrame containing:
        - all grouping columns,
        - the datetime column, shifted by 48 h,
        - a new column with the rolling mean.
    """

    return (
        dfgb[c]
        .rolling(pd.Timedelta(f"{window}h"), min_periods=window, closed="left")
        .mean()
        .reset_index()
        .pipe(
            lambda d: d.assign(
                **{d.columns[0]: d[d.columns[0]] + pd.Timedelta(hours=48)}
            )
        )
        .rename(columns={c: f"{window}h_ma_2d_lag_{c}"})
    )

In [ ]:
# dfgb = df.set_index("datetime").sort_index().groupby(CATEGORICAL_C, as_index=False, observed=False)
# ["target"]

In [ ]:
# get_moving_average(dfgb, 'target', 2).head(10)

In [ ]:
# Drop spring NaNs and impute autumn NaNs with interpolated values
df = train_df.loc[~train_df["datetime"].isin(na_datetimes[1::2])].assign(
    target=lambda x: x["target"].interpolate()
)

# # Add a flag indicating Daylight Saving Time
# df["dst"] = ~(
#     ((df.datetime >= na_datetimes[0]) & (df.datetime < na_datetimes[1]))
#     | ((df.datetime >= na_datetimes[2]) & (df.datetime < na_datetimes[3]))
# )

# estonia_holidays = holidays.EE(years=range(2021, 2024), language='en_US')
# for date, name in estonia_holidays.items():
#     print(date, name)

In [ ]:
# # target lag features 
# for lag in [2, 3, 7]:
#     df = df.merge(
#         add_lag(df[TARGET_C + ['target']], "datetime", lag, 'target'),
#         how="left",
#         on=TARGET_C,
#     )

In [ ]:
# # rolling mean std 48, rolling_mean_96, roll_min_48, roll_max_48
# # delta_48, pct_change_48
# # rolling_mean_temp_48, temp_forecast_horizon

# for r in [24, 24 * 2, 24 * 7]:
#     display(df.set_index("datetime").sort_index().groupby(CATEGORICAL_C, as_index=False, observed=False)["target"].rolling(pd.Timedelta(f"{r}h"), min_periods=r, closed="left").mean().iloc[20:30].assign(new=lambda x: x.index + pd.Timedelta("48 h")).head())

In [ ]:
# df.loc[
#     (df["county"] == "harjumaa")
#     & (df["product_type"] == "combined")
#     & (df["is_business"] == "business")
#     & (df["is_consumption"] == "consumption")
# ].head()

In [ ]:
# (q[CATEGORICAL_C] == q.iloc[0][CATEGORICAL_C]).all(axis=1)

In [ ]:
# df["winter_time"] = np.where(
#     df.dst, df.datetime - pd.Timedelta("1 h"), df.datetime
# )
# for d in na_datetimes:
#     display(
#         df.loc[
#             ((df[CATEGORICAL_C] == df.iloc[0][CATEGORICAL_C]).all(axis=1))
#             & (
#                 df["datetime"].isin(
#                     pd.date_range(
#                         d - pd.Timedelta("1 h"),
#                         d + pd.Timedelta("1 h"),
#                         freq="h",
#                     )
#                 )
#             ),
#             ["datetime", "winter_time", "dst", "target"],
#         ]
#     )

In [ ]:
# # Concat df without duplicated hours (spring) with df with added skipped time (autumn)
# test_df = (
#     pd.concat(
#         [
#             df.loc[~df["datetime"].isin(na_datetimes[1::2])],
#             df.loc[df["datetime"].isin(na_datetimes[::2])].assign(
#                 winter_time=lambda x: x["winter_time"] - pd.Timedelta(" 1h ")
#             ),
#         ]
#     )
#     .sort_values(
#         [
#             "county",
#             "product_type",
#             "is_business",
#             "is_consumption",
#             "winter_time",
#         ]
#     )
#     .reset_index(drop=True)
#     .assign(target=lambda x: x["target"].interpolate())
# )

In [ ]:
df = pd.merge(
    left=df.drop(columns=["prediction_unit_id"]),
    right=client_df.drop(columns=["date"]),
    how='left',
    on=["county", "product_type", "is_business", "data_block_id"],
)

In [ ]:
df = df.merge(
    right=gas_prices_df.drop(columns=["origin_date", "forecast_date"]),
    on=["data_block_id"],
)

In [ ]:
df = df.merge(
    right=electricity_prices_df.drop(
        columns=["origin_datetime", "forecast_datetime"]
    ),
    left_on=["datetime", "data_block_id"],
    right_on=["electricity_datetime", "data_block_id"],
).drop(columns=["electricity_datetime"])

In [ ]:
fp = "f1_"  # 1 Prefix for columns related to the 1 day forecast
df = df.merge(
    avg_weather_data(forecast_weather_df, station_county_mapping).add_prefix(
        fp
    ),
    how="left",
    left_on=["county", "datetime", "data_block_id"],
    right_on=[
        fp + c for c in ["county", "forecast_datetime", "data_block_id"]
    ],
).drop(
    columns=[
        fp + c
        for c in [
            "county",
            "origin_datetime",
            "hours_ahead",
            "forecast_datetime",
            "data_block_id",
        ]
    ]
)

In [ ]:
# Add 2‑day lag of historical data since data for the previous day is
# not fully available (data_block_id increments at 11:00)
hp = "h2_"  # Prefix for columns related to 2 day historical data
hw_df = avg_weather_data(historical_weather_df, station_county_mapping)
hw_df["fully_available_at"] = hw_df["datetime"] + pd.Timedelta("2 d")
df = df.merge(
    hw_df.add_prefix(hp),
    how="left",
    left_on=["county", "datetime"],
    right_on=[hp + c for c in ["county", "fully_available_at"]],
).drop(
    columns=[
        hp + c
        for c in [
            "county",
            "datetime",
            "fully_available_at",
            "data_block_id",
        ]
    ]
)

del hw_df  # Remove temporal groupping df

In [ ]:
# df = df.dropna()

In [ ]:
plt.figure(figsize=(15, 9))
sns.heatmap(
    df.drop(
        columns=[
            "county",
            "product_type",
            "is_business",
            "is_consumption",
            "datetime",
            "data_block_id"
        ]
    ).corr(),
    annot=True,
    fmt=".1f",
    annot_kws={"size": 8}
)
plt.show()

In [ ]:
# Add historical weather data feateurs
# Add sun/radiation/angle feature or combination

## Target

### Distribution

In [ ]:
# Creating target's discrete intervals
bins = 10
plt.figure(figsize=(12, 6))

target_bins_percentage = (
    pd.cut(df.target, bins=bins, precision=0)
    .value_counts()
    .div(len(df))
    .mul(100)
    .round(2)
)

# Due to rounding, the sum of percentages is not exactly 100.0, so
# adjust the first value to ensure the total sum is 100.0
target_bins_percentage[0] += 100 - target_bins_percentage.sum()
target_bins_percentage = [f"{i:.2f}%" for i in target_bins_percentage]

target_max = df.target.max()
ticks = range(0, int(target_max) + 1, int(target_max / bins))

ax = sns.histplot(
    data=df.target,
    bins=bins,
    kde=True,
    linewidth=0.5,
)

# Adding group percentage to the top of each bar
ax.bar_label(ax.containers[0], target_bins_percentage, padding=6, fontsize=11)

plt.xticks(ticks=ticks, rotation=0)

# Using a logarithmic scale for the y-axis for better visualization
# of small quantities of target values
plt.yscale("log")

plt.title(
    f"Histogram of {bins} discrete bins of the target values with KDE-line",
    fontsize=13,
)
plt.xlabel("Target values", fontsize=10)
plt.ylabel("Count, log scale", fontsize=10, rotation=0, labelpad=45)
plt.grid(False, axis="x")
plt.show()

- The target distribution is non-normal and right-skewed, likely because many prosumers are individuals with contracts for their homes, resulting in lower consumption/generation values.
- The first discrete bin contains more values than all the other bins combined, with the KDE line confirming that most values are concentrated below 500 (which corresponds to roughly the left third of the range of the first bin on the x-axis).

In [ ]:
levels = 3  # The number of boxenplot levels

# List with data for additional lines and text started from .25 because
# lower corresponding percentiles are the same

levels_list = [0.25] + np.cumsum(
    [0.5 / pow(2, i) for i in range(levels + 1)]
).tolist()
levels_values = df.target.describe(levels_list)[3:]

plt.figure(figsize=(3, 7.5))
ax = sns.boxenplot(
    df,
    y="target",
    linewidth=0,
    k_depth=levels,
    flier_kws={
        "marker": ".",
        "s": 0.1,
    },
)

ax.set_xlim(ax.get_xlim()[0], ax.get_xlim()[1])

plt.hlines(levels_values.values, 0, ax.get_xlim()[1], "tab:orange", lw=1)

for label, value in levels_values.items():
    plt.text(
        ax.get_xlim()[1] + 0.03,
        value,
        f"{label}: {value:.2f}",
        fontsize=11,
        ha="left",
        va="center",
    )

ax.set_ylim(-0.1, 30_000)
plt.yscale("symlog", linthresh=1)
plt.title(f"Boxenplot of the target values with {levels} levels", fontsize=13)
plt.ylabel("Target values, log scale", fontsize=10, rotation=0, labelpad=65)
plt.yticks(fontsize=11)
plt.show()

- Q<sub>1</sub> ≈ 0.38
- Q<sub>2</sub> ≈ 31.26
- Q<sub>3</sub> ≈ 180.63

There are only two levels on the Q<sub>1</sub> side of the boxenplot (as opposed to three levels on the Q<sub>3</sub> side), which means that there is a huge number of identical values that cannot be separated. That is, two different percentiles (6.25% and 12.5%) have the same value, which equal to the minimum value - 0.

In [ ]:
df["target"].value_counts().head()

Zero values are the most common and occur in more than 15% of the cases.

### Target Over Time

Both classes have equal numbers of entries.

In [ ]:
# Consumption values are multiplied by (-1) for better visualisation
df["modified_target"] = np.where(
    df["is_consumption"] == "consumption",
    df["target"].mul(-1),
    df["target"],
)

In [ ]:
# Adding new time-related features based on datetime

# df["hour"] = df["datetime"].dt.hour.astype("uint8")
# df["day_of_week"] = df["datetime"].dt.day_of_week.astype("uint8")
# df["day"] = df["datetime"].dt.day.astype("uint16")

# df["month"] = df["datetime"].dt.month.astype("int8")
# df["date"] = df["datetime"].dt.date.astype("category")
# df["quarter"] = df["datetime"].dt.quarter.astype("int8")
# df["year"] = df["datetime"].dt.year.astype("uint16")

df["day"] = df["datetime"].dt.day.astype("uint8").astype("category")
df["month"] = df["datetime"].dt.month.astype("uint8").astype("category")
df["year"] = df["datetime"].dt.year.astype("uint16").astype("category")
df["date"] = pd.to_datetime(df[["year", "month", "day"]])

# df["year_week"] = df['datetime'].dt.strftime('%Y-%W').astype("category")
# df["year_month"] = df['datetime'].dt.strftime('%Y-%m').astype("category")

In [ ]:
fig, ax = plt.subplots(figsize=(13, 8))
sns.scatterplot(
    data=df,
    x="datetime",
    y="modified_target",
    hue="is_consumption",
    s=2,
)

target_handles = [
    circle_label("tab:blue", "Consumption"),
    circle_label("tab:orange", "Production"),
]
plt.legend(
    handles=target_handles,
    title="Target type",
    title_fontsize=12,
    bbox_to_anchor=(1, 1),
    loc="upper left",
    borderaxespad=0,
    frameon=False,
    fontsize=11,
)

ax.xaxis.set_major_locator(MonthLocator(interval=2))
ax.yaxis.set_major_locator(MultipleLocator(2000))

plt.title("Target values over time", fontsize=13)
plt.xlabel("Date", fontsize=10)
plt.ylabel("Target", fontsize=10, rotation=0, labelpad=30)
plt.grid(color="grey", linewidth=0.5, alpha=0.5)
plt.show()

Notable observations and assumptions:

- Electricity consumption and production values have been increasing year over year.
- The plot shows a wider range of values for consumption, likely because many production values are 0 or near 0. As a result, target consumption values tend to be higher in terms of descriptive statistics.
- Seasonal cycles: Production values approach zero during winter and increase toward midsummer, while consumption behaves oppositely, peaking in winter and decreasing in summer.
- Weekly cycles: Both consumption and production target values exhibit cyclic patterns, likely tied to working days and weekends.
- A decrease in maximum values and the presence of 'voids' for high consumption values are observed around the New Year holidays, which could suggest a pause in business operations during this period (i.e., target values with `is_business` = True).

Given the high point density near 0 and the prevalence of high values, creating a symlog variant of the scatterplot with values grouped by year-week would be beneficial. Furthermore, using color differentiation by `product_type` instead of `is_consumption` would enhance the visualization.

In [ ]:
fig, ax = plt.subplots(figsize=(13, 8))
sns.scatterplot(
    df.groupby(
        ["year", "month", "day", "is_consumption", "product_type"],
        observed=True,
        as_index=False,
    )["modified_target"]
    .mean()
    .assign(date=lambda d: pd.to_datetime(d[["year", "month", "day"]])),
    x="date",
    y="modified_target",
    hue="product_type",
    s=13,
)

plt.legend(
    title="Contract type",
    title_fontsize=12,
    bbox_to_anchor=(1, 1),
    loc="upper left",
    borderaxespad=0,
    markerscale=3,
    frameon=False,
    fontsize=11,
)

plt.yscale("symlog", linthresh=11)

plt.title(
    "Average energy consumption (below 0) or production (above 0) per "
    "day for each contract type",
    fontsize=13,
)
plt.xlabel("Date", fontsize=10)
plt.ylabel("Target, log scale", fontsize=10, rotation=0, labelpad=30)
plt.grid(alpha=0.3, c="grey")
plt.show()

- Seasonal variations in daily average consumption values are significantly smaller in relative terms (i.e., the percentage change between summer and winter) compared to production. This is likely because energy production by prosumers is severely limited in winter due to low temperatures, shorter daylight hours, and a lower angle of sunlight.
- The lowest daily average consumption values are observed for general_service contracts, followed by fixed, combined, and spot contracts. While there are small overlaps in consumption levels on certain dates, these four series are still visually distinguishable.
- The range of daily average energy production values follows a similar pattern, but with more overlaps: the lowest values are found in general_service, the highest in spot, while fixed and combined contracts fall somewhere in between.

Additionally, it would be useful to analyze production and consumption across different counties.

In [ ]:
g = sns.FacetGrid(
    df.groupby(
        ["year", "month", "day", "is_consumption", "county"],
        observed=True,
        as_index=False,
    )["modified_target"]
    .mean()
    .assign(date=lambda d: pd.to_datetime(d[["year", "month", "day"]])),
    col="county",
    hue="is_consumption",
    col_wrap=8,
    height=1.8,
)

g.map_dataframe(
    sns.scatterplot, x="date", y="modified_target", s=4, edgecolor="none"
)

for ax in g.axes.flat:
    ax.xaxis.set_major_locator(MonthLocator(interval=6))
    ax.xaxis.set_major_formatter(DateFormatter("%m.%y"))
    ax.tick_params(axis="x", rotation=30)
    ax.yaxis.set_major_locator(MultipleLocator(1000))

g.set_axis_labels("Date", "Target")
g.set_titles(col_template="{col_name}")
g.add_legend(
    title="Target type",
    label_order=["production", "consumption"],
    markerscale=4,
    bbox_to_anchor=(0.9, 0.9),
    loc="upper left",
    borderaxespad=0,
    frameon=False,
    fontsize=11,
)
g.legend.get_title().set_fontsize(12)
g.figure.suptitle(
    "Average energy consumption (below 0) or production (above 0) per "
    "week for each county",
    fontsize=13,
)
g.figure.subplots_adjust(top=0.84, wspace=0.15, hspace=0.35)
plt.show()

- Harjumaa and Tartumaa have the highest average energy production and consumption levels, likely due to the presence of large-scale energy consumers and producers in these counties. It is also probable that many consumers in these areas have combined or fixed contract types.
- Some counties, such as Hiiumaa, exhibit low and relatively stable energy production and/or consumption levels.
- Certain counties display abrupt changes in the graphs that do not repeat. For example, Valgamaa experiences a sharp increase in electricity production toward the end of the observation period, Ida-Virumaa sees the emergence of new consumption levels during the first half of the period, and unknown county undergoes a sharp decline in consumption levels (which later returns to a previous level), whereas in Valgamaa, the decline does not reverse. These changes are most likely due to fluctuations in the number of installed solar panels and batteries.
- Seasonal patterns are evident in almost all graphs.

### Time Series Decomposition

[Time Series Gaps](#Time-Series-Gaps)

In [ ]:
def example_df(
    df: pd.DataFrame, c: str, pt: str, isb: str, isc: str
) -> pd.DataFrame:
    """
    Filter DataFrame by county, product_type, is_business,
    is_consumption. And return DataFrame with datetime index and
    target.
    """
    return (
        df[
            (df.county == c)
            & (df.product_type == pt)
            & (df.is_business == isb)
            & (df.is_consumption == isc)
        ][["datetime", "target"]]
        .set_index("datetime")
        .copy()
    )

In [ ]:
# decomposition = seasonal_decompose(
#     ex_df3,
#     model="additive",
#     period=24,
# )
# fig = decomposition.plot()
# fig.set_size_inches((4, 3))
# fig.tight_layout()
# plt.xticks(rotation='vertical')
# plt.show()

In [ ]:
# Selected timeseries
decompose_cat = [
    ["harjumaa", "fixed", "not_business", "consumption"],
    ["ida-virumaa", "spot", "business", "consumption"],
    ["tartumaa", "fixed", "business", "production"],
    ["valgamaa", "spot", "business", "production"],
]

plot_components = ["Observed", "Trend", "Seasonal", "Residual"]

fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(14, 6), sharex="col")

for row, name in enumerate(plot_components):
    axes[row, 0].set_ylabel(name)

for i, categories in enumerate(decompose_cat):
    decomp = seasonal_decompose(
        # 24 hour period
        example_df(df, *categories), model="additive", period=24
    )
    axes[0, i].set_title(categories[0], fontsize=11)
    axes[0, i].plot(decomp.observed)
    axes[1, i].plot(decomp.trend)
    axes[2, i].plot(decomp.seasonal)
    axes[3, i].plot(decomp.resid)

    for ax in axes[:, i]:
        ax.tick_params(axis="x", rotation=45)

fig.tight_layout()
fig.subplots_adjust(top=0.875, wspace=0.25, hspace=0.3)
fig.suptitle(
    "Seasonal decomposition for different timeseries using moving averages",
    fontsize=13,
)
plt.show()

The removal of the 24-hour component from the timeseries is not sufficient; both the trend and residual components still exhibit cyclic patterns (including both low-frequency and high-frequency cycles).

In [ ]:
periods = [24, 24 * 7, 24 * 30]
mstl = MSTL(example_df(df, *decompose_cat[0]), periods=periods)
res = mstl.fit()

fig = res.plot()
fig.set_size_inches((16, 12))
fig.tight_layout()
# plt.xticks(rotation='vertical')
plt.show()

Adding only weekly and monthly cycles is insufficient: the trend component still exhibits at least annual seasonality, but with fewer than two full years of data, MSTL cannot reliably estimate yearly effects. The residuals also display persistent cyclic patterns. Moreover, the amplitude of the weekly and monthly seasonal components increases with the trend - suggesting that a multiplicative model or alternative methods would be more appropriate in this case.

In [ ]:
# m = Prophet(
#     yearly_seasonality=True,
#     weekly_seasonality=True,
#     daily_seasonality=True,
#     seasonality_mode='multiplicative',
# )
# m.fit(example_df(df, *decompose_cat[2]).reset_index().rename(columns={"datetime": "ds", "target": "y"}))

# future = m.make_future_dataframe(periods=24*60, freq='h')
# forecast = m.predict(future)

# fig1 = m.plot(forecast)


# fig2 = m.plot_components(forecast)
# plt.tight_layout()
# plt.show()

## Client features

In [ ]:
capacity_eic = (
    df.groupby(
        ["year", "month", "county"],
        observed=True,
        as_index=False,
    )
    .agg(
        installed_capacity=("installed_capacity", "mean"),
        eic_count=("eic_count", "mean"),
    )
    .assign(date=lambda x: pd.to_datetime(x[["year", "month"]].assign(day=15)))
    .drop(columns=["year", "month"])
)

In [ ]:
def twin_scatterplot(data, x, y, color, label, **kwargs):
    ax = plt.gca()
    twin_ax = ax.twinx()
    sns.scatterplot(
        data=data, x=x, y=y, color=color, ax=twin_ax, s=22, **kwargs
    )
    twin_ax.tick_params(axis="y", colors=color)
    twin_ax.yaxis.label.set_color(color)
    facet_idx = ax.get_subplotspec().colspan.start
    if (facet_idx + 1) % n_cols == 0:
        twin_ax.set_ylabel(label, fontsize=11)
    else:
        twin_ax.set_ylabel("")
    twin_axes.append(twin_ax)


twin_axes = []
n_cols = 8

g = sns.FacetGrid(
    capacity_eic,
    col="county",
    col_wrap=n_cols,
    sharex=True,
    sharey="row",
    height=2.5,
    aspect=1,
)

g.map_dataframe(
    sns.lineplot,
    x="date",
    y="installed_capacity",
    color="tab:blue",
    label="Installed_capacity"
)
g.map_dataframe(
    twin_scatterplot,
    x="date",
    y="eic_count",
    color="tab:orange",
    label="EIC count",
)

for ax in g.axes.flat:
    ax.set_ylabel("Installed capacity", fontsize=11, color="tab:blue")
    ax.xaxis.set_major_locator(MonthLocator(interval=6))
    ax.tick_params(axis="x", rotation=30)
    ax.tick_params(axis="y", colors="tab:blue")


for i in range(0, len(twin_axes), n_cols):
    row_axes = twin_axes[i : i + n_cols]
    ylims = [ax.get_ylim() for ax in row_axes if len(ax.collections) > 0]
    if ylims:
        ymin = min(lim[0] for lim in ylims)
        ymax = max(lim[1] for lim in ylims)
        for ax in row_axes:
            ax.set_ylim(ymin, ymax)

for idx, ax in enumerate(g.axes.flat):
    if idx % 8 != 0:
        ax.tick_params(
            axis="y",
            which="both",
            left=False,
            labelleft=False,
        )

for idx, twin_ax in enumerate(twin_axes):
    if (idx + 1) % 8 != 0:
        twin_ax.set_yticklabels([])
        twin_ax.tick_params(axis="y", which="both", length=0)
    else:
        label = ax.get_ylabel()
        ax.set_ylabel(label, fontsize=11)

legend_elements = [
    circle_label("tab:blue", "Installed capacity"),
    circle_label("tab:orange", "EIC count"),
]

g.figure.legend(
    title="Client feature",
    title_fontsize=12,
    handles=legend_elements,
    loc="upper right",
    frameon=False,
    bbox_to_anchor=(1, 0.95),
    fontsize=11,
)

g.figure.suptitle(
    "Average installed capacity and EIC count per month for each county",
    fontsize=13,
)
sns.despine(fig=g.fig, left=True, bottom=True, right=True, top=True)
g.figure.subplots_adjust(top=0.88,right=0.87, wspace=0.05, hspace=0.225)
g.set_titles(col_template="{col_name}", size=12)
plt.show()

### Compare different aggregations of weather forecast - historical weather.
### Feature Engineering

In [ ]:
# df_hours = df.copy()

# hours_ago = (
#     [i for i in range(1, 25)]
#     + [24 * i for i in range(2, 8)]
#     + [168 * i for i in range(2, 9)]
#     + [672 * i for i in range(3, 13)]
# )
# for h in hours_ago:
#     df_hours[f"tm_{h}h"] = df_hours["modified_target"].shift(h)
# df_hours.info()

In [ ]:
# df["tm_1h"] = df["modified_target"].shift(1)
# def add_lags(df):
#     target_map = df['PJME_MW'].to_dict()
#     df['lag1'] = (df.index - pd.Timedelta('364 days')).map(target_map)
#     df['lag2'] = (df.index - pd.Timedelta('728 days')).map(target_map)
#     df['lag3'] = (df.index - pd.Timedelta('1092 days')).map(target_map)
#     return df
# df_label = pd.get_dummies(df_label, drop_first=True)
# df_label.info()

In [ ]:
# X = df_label.drop(
#     columns=[
#         "target",
#         "data_block_id",
#         "row_id",
#         "prediction_unit_id",
#         "modified_target",
#     ],
#     axis=1,
# )

# y = df_label["modified_target"].values

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.20, random_state=RAND
# )

# st = StandardScaler()
# X_train_std = st.fit_transform(X_train)
# X_test_std = st.transform(X_test)

In [ ]:
# def r2_adjusted(
#     y_true: np.ndarray, y_pred: np.ndarray, X_test: np.ndarray | int
# ) -> float:
#     """Коэффициент детерминации (множественная регрессия)"""
#     N_objects = len(y_true)

#     if isinstance(X_test, np.ndarray):
#         N_features = X_test.shape[1]
#     else:
#         N_features = X_test

#     #     N_features = X_test.shape[1]
#     r2 = r2_score(y_true, y_pred)
#     return 1 - (1 - r2) * (N_objects - 1) / (N_objects - N_features - 1)


# def mpe(y_true: np.ndarray, y_pred: np.ndarray) -> float:
#     """Mean percentage error"""
#     return np.mean((y_true - y_pred) / y_true, axis=0) * 100


# def mape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
#     """Mean absolute percentage error"""
#     return np.mean(np.abs((y_pred - y_true) / y_true), axis=0) * 100


# def wape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
#     """Weighted Absolute Percent Error"""
#     return np.sum(np.abs(y_pred - y_true)) / np.sum(y_true) * 100


# def huber_loss(
#     y_true: np.ndarray | pd.DataFrame,
#     y_pred: np.ndarray | pd.DataFrame,
#     delta: float = 1.345,
# ):
#     """Функция ошибки Хьюбера"""

#     if isinstance(y_true, pd.DataFrame):
#         y_true = y_true.squeeze().to_numpy()
#     if isinstance(y_pred, pd.DataFrame):
#         y_pred = y_pred.squeeze().to_numpy()

#     assert len(y_true) == len(y_pred), "Разные размеры данных"
#     huber_sum = 0
#     for i in range(len(y_true)):
#         if abs(y_true[i] - y_pred[i]) <= delta:
#             huber_sum += 0.5 * (y_true[i] - y_pred[i]) ** 2
#         else:
#             huber_sum += delta * (abs(y_true[i] - y_pred[i]) - 0.5 * delta)
#     huber_sum /= len(y_true)
#     return huber_sum


# def logcosh(y_true: np.ndarray, y_pred: np.ndarray):
#     """функция ошибки Лог-Кош"""
#     return np.sum(np.log(np.cosh(y_true - y_pred)))


# def rmsle(y_true: np.ndarray, y_pred: np.ndarray) -> np.float64:
#     """
#     Root Mean Squared Log Error (RMSLE) metric
#     Логарифмическая ошибка средней квадратичной ошибки
#     """
#     try:
#         return np.sqrt(mean_squared_log_error(y_true, y_pred))
#     except:
#         return None


# def get_metrics(
#     y_test: np.ndarray,
#     y_pred: np.ndarray,
#     X_test: np.ndarray,
#     name: str = None,
#     delta: float = 1.345,
# ):
#     """Генерация таблицы с метриками"""
#     df_metrics = pd.DataFrame()
#     df_metrics["model"] = [name]

#     df_metrics["MAE"] = mean_absolute_error(y_test, y_pred)
#     df_metrics["MSE"] = mean_squared_error(y_test, y_pred)
#     df_metrics["Huber_loss"] = huber_loss(y_test, y_pred, delta)
#     df_metrics["Logcosh"] = logcosh(y_test, y_pred)
#     df_metrics["RMSE"] = np.sqrt(mean_squared_error(y_test, y_pred))
#     df_metrics["RMSLE"] = rmsle(y_test, y_pred)
#     df_metrics["R2 adjusted"] = r2_adjusted(y_test, y_pred, X_test)
#     df_metrics["MPE_%"] = mpe(y_test, y_pred)
#     df_metrics["MAPE_%"] = mape(y_test, y_pred)
#     df_metrics["WAPE_%"] = wape(y_test, y_pred)

#     return df_metrics

In [ ]:
# df = df.sort_index().sort_values(
#     ["county", "is_business", "product_type", "is_consumption"],
#     kind="mergesort",
# )
# df.sort_index()
# tss = TimeSeriesSplit(n_splits=3, test_size=300_000)

# a = {}
# for i in range(1000):
#     x = 0
#     for j in range(20):
#         x += np.random.choice([-1, 1])
#     a[x] = a.get(x, 0) + 1


# sns.barplot(x=list(a.keys()), y=list(a.values()));

In [ ]:
# min(data.keys())
# max(data.keys())
# len(data.keys())
# {k: 0 for (k, 0) in range(min(data.keys()), max(data.keys())) if not in data.keys()}
# {k: v*2 for (k,v) in dict1.items()}
# {key:value for (key,value) in dictonary.items()}
# zip()

# for train_idx, val_idx in tss.split(df):
# print('1 train:', train_idx)
# display(df.iloc[train_idx].tail(5))
# print('1 val:', val_idx)

In [ ]:
# tss = TimeSeriesSplit(n_splits=3, test_size=300_000)
# df = df.sort_index()

# fig, axs = plt.subplots(3, 1, figsize=(10, 10), sharex=True)
# fold = 0

# for train_idx, val_idx in tss.split(df):
#     train = df.iloc[train_idx]
#     test = df.iloc[val_idx]
#     train["modified_target"].plot(
#         ax=axs[fold],
#         label="Training Set",
#         title=f"Data Train/Test Split Fold {fold}",
#     )
#     test["modified_target"].plot(ax=axs[fold], label="Test Set")
#     axs[fold].axvline(test.index.min(), color="black", ls="--")
#     fold += 1
# plt.show()

In [ ]:
# fold = 0
# preds = []
# scores = []
# for train_idx, val_idx in tss.split(df):
#     train = df.iloc[train_idx]
#     test = df.iloc[val_idx]

#     reg = XGBRegressor(
#         n_estimators=2000,
#         early_stopping_rounds=50,
#         objective="reg:squarederror",
#         enable_categorical=True,
#         eval_metric="mae",
#         # max_depth=3,
#         learning_rate=0.01,
#         random_state=RAND,
#     )
#     FEATURES = [
#         "county",
#         "is_business",
#         "product_type",
#         # 'target',
#         "is_consumption",
#         # 'data_block_id',
#         # 'row_id',
#         # 'prediction_unit_id',
#         "hour",
#         "day_of_week",
#         "day",
#         "week_of_year",
#         "month",
#         "quarter",
#         "year",
#         # 'modified_target',
#     ]
#     TARGET = "modified_target"

#     X_train = train[FEATURES]
#     y_train = train[TARGET]

#     X_test = test[FEATURES]
#     y_test = test[TARGET]

#     reg.fit(
#         X_train,
#         y_train,
#         eval_set=[(X_train, y_train), (X_test, y_test)],
#         verbose=20,
#     )

#     y_pred = reg.predict(X_test)
#     preds.append(y_pred)
#     score = np.sqrt(mean_squared_error(y_test, y_pred))
#     scores.append(score)